In [8]:
import pandas as pd
import json
import seaborn as sns
import numpy as np


import matplotlib.pyplot as plt
from openai import OpenAI
from langchain_openai import OpenAIEmbeddings

In [9]:
sns.set_theme()
plt.rcParams["font.sans-serif"] = ["SimHei"]
plt.rcParams["axes.unicode_minus"] = False

api_key = "your_api_key"
api_base = "your_api_base"
client = OpenAI(api_key=api_key, base_url=api_base)

# 1. 向量表示
## 导入数据

In [40]:
data1 = pd.read_csv("../data/train_dataset.csv")
data1.head()

,joke,label,len_joke,标签,pinyin
0,如果你有一百万莎士比亚，他们会像猴子一样写作吗？,1,24,一般幽默,ru2 guo3 ni3 you3 yi1 bai3 wan4 sha1 shi4 bi3 ...
1,本村有一小伙，在武校上了四五年了， 今年他爹不让他去了，换普通高中了， 因为过年亲戚来了喝了...,1,91,一般幽默,ben3 cun1 you3 yi1 xiao3 huo3 ， zai4 wu3 xiao4...
2,一天，我刚洗完澡回屋准备穿衣服，刚要穿，房东的女儿倩倩闯进来了，看到我没穿衣服，“啊！”的一...,1,124,一般幽默,yi1 tian1 ， wo3 gang1 xi3 wan2 zao3 hui2 wu1 z...
3,如果我记得你的名字，我会问你把钥匙放在哪里。,1,22,一般幽默,ru2 guo3 wo3 ji4 de2 ni3 de5 ming2 zi4 ， wo3 h...
4,我爸爸和几个同事准备去美国，在超市买随身带的东西呢，这时，超市里一个美国人过来用标准的济南话...,1,125,一般幽默,wo3 ba4 ba4 he2 ji1 ge4 tong2 shi4 zhun3 bei4 ...


In [41]:
data2 = pd.read_csv("../data/test_dataset.csv")
data2.head()

,joke,label,len_joke,标签,pinyin
0,车上遇见一对双胞胎姑娘。穿的那个清凉啊，笑的那个甜啊……忍不住要去搭讪。 为不表现的轻浮和唐...,2,83,强幽默,che1 shang4 yu4 jian4 yi1 dui4 shuang1 bao1 ta...
1,图书馆自习室每个桌面上都摆满了占座的书，很是让其他人气愤。 某生不占座没有固定的座位，一日去...,0,192,弱幽默,tu2 shu1 guan3 zi4 xi2 shi4 mei3 ge4 zhuo1 mia...
2,一个报童在大街上高声叫卖：骇人听闻的诈骗案，受害者多达82人！ 某行人连忙上前买一份。可是，...,2,102,强幽默,yi1 ge4 bao4 tong2 zai4 da4 jie1 shang4 gao1 s...
3,法官对被告说：你不但偷钱，还拿了表，戒指和珍珠。 被告说：是的，法官先生，人们不是常说‘光有...,2,56,强幽默,fa3 guan1 dui4 bei4 gao4 shuo1 ： ni3 bu4 dan4 ...
4,大人，原告在法庭上说，这个人同我一起生活了几天，答应同我结婚，可是后来他同别的女人结了婚。他...,1,140,一般幽默,da4 ren2 ， yuan2 gao4 zai4 fa3 ting2 shang4 sh...


## 使用向量嵌入模型

In [49]:
# 初始化嵌入模型
embeddings = OpenAIEmbeddings(
    api_key=api_key, base_url=api_base, model="text-embedding-3-large", dimensions=1024
)

In [53]:
embeddings_list1 = embeddings.embed_documents(data1["joke"].to_list())

In [54]:
data1["text_embedding"] = embeddings_list1
data1[["joke", "text_embedding", "label"]].to_csv(
    "../data/train_dataset_embedding.csv", index=False
)

In [50]:
embeddings_list2 = embeddings.embed_documents(data2["joke"].to_list())

In [52]:
data2["text_embedding"] = embeddings_list2
data2[["joke", "text_embedding", "label"]].to_csv(
    "../data/test_dataset_embedding.csv", index=False
)

In [56]:
#重新读取时list类型已经变成str类型了
data3 = pd.read_csv("../data/test_dataset_embedding.csv")
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3130 entries, 0 to 3129
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   joke            3130 non-null   object
 1   text_embedding  3130 non-null   object
 2   label           3130 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 73.5+ KB


In [60]:
# str重新变为list
embeddings_list = data3["text_embedding"].apply(lambda x: json.loads(x))

# 2. 向量检索

## 寻找topk向量

In [7]:
data1 = pd.read_csv("../data/train_dataset_embedding.csv")
train_embeddings = data1["text_embedding"].apply(lambda x: json.loads(x))
data2 = pd.read_csv("../data/test_dataset_embedding.csv")
test_embeddings = data2["text_embedding"].apply(lambda x: json.loads(x))

In [10]:
def cosine_similarity_matrix(A, B):
    # 计算矩阵A中每个向量与矩阵B中每个向量的余弦相似性
    dot_products = np.dot(A, B.T)
    norms_A = np.linalg.norm(A, axis=1)
    norms_B = np.linalg.norm(B, axis=1)
    similarities = dot_products / np.outer(norms_A, norms_B)
    return similarities

In [15]:
A = np.array(test_embeddings.to_list())
B = np.array(train_embeddings.to_list())
similarities = cosine_similarity_matrix(A, B)

In [12]:
print(similarities.shape)

(3130, 12521)


In [37]:
topk_similarities = np.argsort(-similarities)

In [44]:
print(data2["joke"].iloc[1])
print(data1["joke"].iloc[topk_similarities[1][0]])

图书馆自习室每个桌面上都摆满了占座的书，很是让其他人气愤。 某生不占座没有固定的座位，一日去自习，练习时发现忘了带草稿本，恰巧桌面那堆书上有一个崭新的练习本，于是打算撕下一张用用。 刚想撕，看见书堆上有张贴纸，上书：“证明你的人品，一切东西请勿乱动！” 该生沉思了会儿，就在贴纸上回了一句话：“你的练习本，我不撕，证明我的人品好；我撕了，证明你的人品不好！事实证明你的人品真的不好！”
快要期末考试了，有个学生想作弊，就提前来到考场，想在课桌上写点小抄，谁知那张课桌上已密密麻麻写满了歌词，这学生大怒，于是在桌上写道：“哥们儿，给我留点儿地方做小抄好吗？” 第二天，有人在课桌上回复：“哥们儿，不好意思，这是我创作并能发表作品的最后一块园地了……” 第三天，这张课桌上又添了一句话：“最烦你们这些人了，搞得我每次上完课睡觉起来，脸上不是考试答案就是歌词！”
